In [1]:
import findspark
findspark.init()
import os
import pyspark
import sys
from pyspark.sql import SparkSession

In [2]:
## Start Spark Session
spark = SparkSession.builder.appName('app_name') \
        .config('spark.jars.packages','org.apache.hudi:hudi-spark3.3-bundle_2.12:0.13.1') \
        .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
        .config('spark.sql.extensions','org.apache.spark.sql.hudi.HoodieSparkSessionExtension') \
        .config('spark.sql.catalog.spark_catalog','org.apache.spark.sql.hudi.catalog.HoodieCatalog') \
        .getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/opt/anaconda3/envs/spark_hudi/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anshumanr/.ivy2/cache
The jars for the packages stored in: /Users/anshumanr/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-97e927cc-185a-414e-bc6e-a7e6d1d37e45;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 in central
:: resolution report :: resolve 85ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submi

23/09/01 13:07:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/01 13:07:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark Running


## Creating Partition Table

In [3]:
tablename='accounts_partition'
basePath = "/Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/accounts_partition"            

In [4]:
df = spark.read.format('parquet').options(inferSchema = 'True',header='True').load("/Users/anshumanr/Downloads/data/accounts.parquet")

In [5]:
hudi_options = {
    'hoodie.table.name': tablename,
    'hoodie.datasource.write.recordkey.field': 'account_id',
    'hoodie.datasource.write.partitionpath.field': 'account_type',
    'hoodie.datasource.write.table.name': tablename,
    'hoodie.datasource.write.operation': 'upsert',
    'hoodie.datasource.write.precombine.field':'open_date'
}

In [6]:
df.write.format("hudi"). \
    options(**hudi_options). \
    mode("overwrite"). \
    save(basePath)

23/09/01 13:07:13 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/09/01 13:07:13 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/09/01 13:07:13 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file
23/09/01 13:07:13 WARN HoodieSparkSqlWriter$: hoodie table at /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/accounts_partition already exists. Deleting existing data & overwriting with new data.
23/09/01 13:07:14 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/accounts_partition/.hoodie/metadata


23/09/01 13:07:28 WARN MemoryStore: Not enough space to cache rdd_22_4 in memory! (computed 5.1 MiB so far)
23/09/01 13:07:28 WARN BlockManager: Persisting block rdd_22_4 to disk instead.


23/09/01 13:07:41 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_39_4 in memory.
23/09/01 13:07:41 WARN MemoryStore: Not enough space to cache rdd_39_4 in memory! (computed 0.0 B so far)
23/09/01 13:07:41 WARN BlockManager: Persisting block rdd_39_4 to disk instead.
23/09/01 13:07:41 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_39_3 in memory.
23/09/01 13:07:41 WARN MemoryStore: Not enough space to cache rdd_39_3 in memory! (computed 0.0 B so far)
23/09/01 13:07:41 WARN BlockManager: Persisting block rdd_39_3 to disk instead.
23/09/01 13:07:41 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_39_2 in memory.
23/09/01 13:07:41 WARN MemoryStore: Not enough space to cache rdd_39_2 in memory! (computed 0.0 B so far)
23/09/01 13:07:41 WARN BlockManager: Persisting block rdd_39_2 to disk instead.
23/09/01 13:07:42 WARN MemoryStore: Not eno

# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]
23/09/01 13:08:19 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-hbase.properties,hadoop-metrics2.properties


## Creating Non-Partitioned table from same same file

In [7]:
tablename='accounts_non_partition'
basePath = "/Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/accounts_non_partition" 

In [8]:
df = spark.read.format('parquet').options(inferSchema = 'True',header='True').load("/Users/anshumanr/Downloads/data/accounts.parquet")

In [9]:
hudi_options = {
    'hoodie.table.name': tablename,
    'hoodie.datasource.write.recordkey.field': 'account_id',
    'hoodie.datasource.write.table.name': tablename,
    'hoodie.datasource.write.operation': 'upsert',
    'hoodie.datasource.write.precombine.field':'open_date'
}

In [10]:
df.write.format("hudi").options(**hudi_options).mode("overwrite").save(basePath)

23/09/01 13:08:20 WARN HoodieSparkSqlWriter$: hoodie table at /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/accounts_non_partition already exists. Deleting existing data & overwriting with new data.
23/09/01 13:08:20 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/accounts_non_partition/.hoodie/metadata


23/09/01 13:08:30 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_96_4 in memory.
23/09/01 13:08:30 WARN MemoryStore: Not enough space to cache rdd_96_4 in memory! (computed 0.0 B so far)
23/09/01 13:08:30 WARN BlockManager: Persisting block rdd_96_4 to disk instead.
23/09/01 13:08:30 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_96_7 in memory.
23/09/01 13:08:30 WARN MemoryStore: Not enough space to cache rdd_96_7 in memory! (computed 0.0 B so far)
23/09/01 13:08:30 WARN BlockManager: Persisting block rdd_96_7 to disk instead.
23/09/01 13:08:30 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_96_6 in memory.
23/09/01 13:08:30 WARN MemoryStore: Not enough space to cache rdd_96_6 in memory! (computed 0.0 B so far)
23/09/01 13:08:30 WARN BlockManager: Persisting block rdd_96_6 to disk instead.
23/09/01 13:08:30 WARN MemoryStore: Failed 